In [36]:
import pickle
import numpy as np
import pandas as pd

In [37]:
%store -r test

# Take the name of the different classes (e.g. '0', '1', '2', ...)
classes = test.groupby(['label'])['label'].unique().index.to_list()

In [38]:
test

,label,x,y,speed,strokes,angles,angles_tangent
795,2,"[345.0, 344.0, 344.0, 347.0, 350.0, 364.0, 369...","[303.0, 301.0, 297.0, 289.0, 284.0, 265.0, 257...",0.849,1,"[-2.03, -1.57, -1.21, -1.03, -0.94, -1.01, -1....","[2.8109, 3.0702, 3.0936, 2.9991, 2.8535, 2.661..."
319,8,"[785.0, 744.0, 711.0, 675.0, 639.0, 607.0, 579...","[199.0, 193.0, 191.0, 191.0, 195.0, 202.0, 212...",1.084,1,"[-3.0, -3.08, 3.14, 3.03, 2.93, 2.8, 2.65, 2.4...","[2.6987, 2.6248, 2.5304, 2.4012, 2.2604, 2.107..."
382,2,"[492.0, 483.0, 474.0, 460.0, 440.0, 429.0, 392...","[199.0, 213.0, 229.0, 248.0, 267.0, 275.0, 295...",1.032,1,"[2.14, 2.08, 2.21, 2.38, 2.51, 2.65, 3.14, -2....","[2.3934, 2.2028, 1.1252, 0.8448, 0.8606, 0.859..."
737,9,"[707.0, 681.0, 650.0, 615.0, 559.0, 540.0, 482...","[81.0, 72.0, 65.0, 61.0, 59.0, 59.0, 67.0, 72....",0.927,1,"[-2.81, -2.92, -3.03, -3.11, 3.14, 3.0, 2.88, ...","[2.7677, 2.7722, 2.7553, 2.7324, 2.7486, 2.681..."
422,2,"[514.0, 529.0, 552.0, 582.0, 618.0, 653.0, 686...","[184.0, 168.0, 151.0, 136.0, 125.0, 119.0, 118...",1.939,1,"[-0.82, -0.64, -0.46, -0.3, -0.17, -0.03, 0.18...","[2.541, 2.4347, 2.281, 2.1718, 2.0967, 2.1447,..."
...,...,...,...,...,...,...,...
211,0,"[762.0, 712.0, 677.0, 643.0, 611.0, 578.0, 535...","[101.0, 131.0, 162.0, 198.0, 238.0, 281.0, 349...",1.659,1,"[2.6, 2.42, 2.33, 2.25, 2.23, 2.13, 2.08, 2.0,...","[2.8502, 2.8683, 2.7854, 2.6799, 2.5227, 2.391..."
691,7,"[715.0, 774.0, 794.0, 832.0, 867.0, 893.0, 905...","[92.0, 89.0, 88.0, 86.0, 85.0, 86.0, 90.0, 96....",3.368,1,"[-0.05, -0.05, -0.05, -0.03, 0.04, 0.32, 0.98,...","[2.6319, 1.6287, 1.0872, 1.0533, 1.3416, 2.365..."
81,0,"[631.0, 622.0, 616.0, 613.0, 604.0, 597.0, 589...","[48.0, 57.0, 63.0, 66.0, 77.0, 85.0, 94.0, 98....",0.511,1,"[2.36, 2.36, 2.36, 2.26, 2.29, 2.3, 2.36, 2.36...","[3.1353, 3.1171, 3.0999, 3.1275, 3.0326, 3.010..."
768,0,"[602.0, 599.0, 592.0, 584.0, 568.0, 557.0, 546...","[260.0, 259.0, 265.0, 273.0, 287.0, 298.0, 312...",1.032,1,"[-2.82, 2.43, 2.36, 2.42, 2.36, 2.24, 2.2, 2.1...","[2.9086, 2.932, 2.9228, 2.8691, 2.8554, 2.8281..."


In [39]:
predicted_test = test.copy()

feature = 'angles_tangent'

models = []
for n_class in classes: # Can be thrown an expection beacuse this is the labels of the test set and not of the models (train set)
    model = pickle.load(open('models/'+str(n_class)+'hmm_model.pkl', 'rb'))
    models.append(model)

log_likelihoods = pd.DataFrame()
log_likelihoods['predicted_class'] = classes
log_like_max = []
top_3_loglikelihood = []
top_3_predicted_classes = []
predicted_class = []
is_predicted = []

for i in range(len(test)):

    true_label = test['label'].iloc[i]
    list_log_likelihoods = []
    for model in models:
        
        angle = test[feature].iloc[i]
        X = [[one] for one in angle]
        list_log_likelihoods.append(model.score(X))
        pass

    log_likelihoods['log_likelihood'] = list_log_likelihoods
    log_likelihoods['true_class'] = true_label

    # Compute top 1 prediction
    max_log_likelihood = log_likelihoods.loc[log_likelihoods['log_likelihood'].idxmax()]
    log_like_max.append(max_log_likelihood['log_likelihood'])
    predicted_class.append(max_log_likelihood['predicted_class'])

    if max_log_likelihood['predicted_class'] == max_log_likelihood['true_class']:
        is_predicted.append(True)
    else:
        is_predicted.append(False)

    # Compute top 3 predictions
    top_3_loglikelihood.append(sorted(list_log_likelihoods, reverse=True)[:3])
    top1, top2, top3 = sorted(list_log_likelihoods, reverse=True)[:3]
    top_3_list = []
    top_3_list.append(log_likelihoods.loc[log_likelihoods['log_likelihood'] == top1]['predicted_class'].values[0])
    top_3_list.append(log_likelihoods.loc[log_likelihoods['log_likelihood'] == top2]['predicted_class'].values[0])
    top_3_list.append(log_likelihoods.loc[log_likelihoods['log_likelihood'] == top3]['predicted_class'].values[0])
    top_3_predicted_classes.append(top_3_list)




predicted_test["predicted_class"] = predicted_class
predicted_test["log_likelihood"] = log_like_max
predicted_test["is_predicted"] = is_predicted
predicted_test["top_3_predicted"] = top_3_predicted_classes
predicted_test["top_3_loglikelihood"] = top_3_loglikelihood

predicted_test

# model_0.score_samples(angles_0)

,label,x,y,speed,strokes,angles,angles_tangent,predicted_class,log_likelihood,is_predicted,top_3_predicted,top_3_loglikelihood
795,2,"[345.0, 344.0, 344.0, 347.0, 350.0, 364.0, 369...","[303.0, 301.0, 297.0, 289.0, 284.0, 265.0, 257...",0.849,1,"[-2.03, -1.57, -1.21, -1.03, -0.94, -1.01, -1....","[2.8109, 3.0702, 3.0936, 2.9991, 2.8535, 2.661...",2.0,-381.144489,True,"[2, 3, 8]","[-381.1444887144453, -387.2865939327944, -388...."
319,8,"[785.0, 744.0, 711.0, 675.0, 639.0, 607.0, 579...","[199.0, 193.0, 191.0, 191.0, 195.0, 202.0, 212...",1.084,1,"[-3.0, -3.08, 3.14, 3.03, 2.93, 2.8, 2.65, 2.4...","[2.6987, 2.6248, 2.5304, 2.4012, 2.2604, 2.107...",8.0,-296.643158,True,"[8, 9, 2]","[-296.6431584812086, -303.4264087027308, -304...."
382,2,"[492.0, 483.0, 474.0, 460.0, 440.0, 429.0, 392...","[199.0, 213.0, 229.0, 248.0, 267.0, 275.0, 295...",1.032,1,"[2.14, 2.08, 2.21, 2.38, 2.51, 2.65, 3.14, -2....","[2.3934, 2.2028, 1.1252, 0.8448, 0.8606, 0.859...",6.0,-242.350187,False,"[6, 2, 8]","[-242.35018663061913, -242.64855775587657, -24..."
737,9,"[707.0, 681.0, 650.0, 615.0, 559.0, 540.0, 482...","[81.0, 72.0, 65.0, 61.0, 59.0, 59.0, 67.0, 72....",0.927,1,"[-2.81, -2.92, -3.03, -3.11, 3.14, 3.0, 2.88, ...","[2.7677, 2.7722, 2.7553, 2.7324, 2.7486, 2.681...",0.0,-384.221634,False,"[0, 9, 6]","[-384.2216340263533, -384.60754966049564, -386..."
422,2,"[514.0, 529.0, 552.0, 582.0, 618.0, 653.0, 686...","[184.0, 168.0, 151.0, 136.0, 125.0, 119.0, 118...",1.939,1,"[-0.82, -0.64, -0.46, -0.3, -0.17, -0.03, 0.18...","[2.541, 2.4347, 2.281, 2.1718, 2.0967, 2.1447,...",6.0,-190.953680,False,"[6, 3, 8]","[-190.95368016241696, -191.60329745507116, -19..."
...,...,...,...,...,...,...,...,...,...,...,...,...
211,0,"[762.0, 712.0, 677.0, 643.0, 611.0, 578.0, 535...","[101.0, 131.0, 162.0, 198.0, 238.0, 281.0, 349...",1.659,1,"[2.6, 2.42, 2.33, 2.25, 2.23, 2.13, 2.08, 2.0,...","[2.8502, 2.8683, 2.7854, 2.6799, 2.5227, 2.391...",2.0,-185.408238,False,"[2, 3, 0]","[-185.40823761030498, -187.16943874042013, -19..."
691,7,"[715.0, 774.0, 794.0, 832.0, 867.0, 893.0, 905...","[92.0, 89.0, 88.0, 86.0, 85.0, 86.0, 90.0, 96....",3.368,1,"[-0.05, -0.05, -0.05, -0.03, 0.04, 0.32, 0.98,...","[2.6319, 1.6287, 1.0872, 1.0533, 1.3416, 2.365...",5.0,-108.527493,False,"[5, 8, 6]","[-108.52749286977082, -108.85394679197552, -11..."
81,0,"[631.0, 622.0, 616.0, 613.0, 604.0, 597.0, 589...","[48.0, 57.0, 63.0, 66.0, 77.0, 85.0, 94.0, 98....",0.511,1,"[2.36, 2.36, 2.36, 2.26, 2.29, 2.3, 2.36, 2.36...","[3.1353, 3.1171, 3.0999, 3.1275, 3.0326, 3.010...",7.0,-665.192430,False,"[7, 6, 9]","[-665.1924304730965, -669.499902156228, -677.3..."
768,0,"[602.0, 599.0, 592.0, 584.0, 568.0, 557.0, 546...","[260.0, 259.0, 265.0, 273.0, 287.0, 298.0, 312...",1.032,1,"[-2.82, 2.43, 2.36, 2.42, 2.36, 2.24, 2.2, 2.1...","[2.9086, 2.932, 2.9228, 2.8691, 2.8554, 2.8281...",5.0,-220.343579,False,"[5, 2, 7]","[-220.34357869485345, -226.17786879150893, -22..."


In [40]:
# Calculate top 3 accuracy
sum_top_3 = 0
for idx, row in predicted_test.iterrows():
    for three_likelihood in row["top_3_predicted"]:
        if(row["label"] == three_likelihood):
            sum_top_3 += 1
            pass

accuracy = sum(predicted_test["is_predicted"])/len(predicted_test["is_predicted"])
print("Accuracy: ", accuracy)
accuracy_top_3 = sum_top_3/len(predicted_test["label"])
print("Accuracy top 3: ", accuracy_top_3)

Accuracy:  0.33714285714285713
Accuracy top 3:  0.64
